In [15]:
import pympi
import os
import math

In [16]:
def get_token(token):
    return token
    
    if token[:8] == "FBUOY:DS":
        split = token.split(':')
        if len(split) > 2:
            token = token.split(':')[2]
    if token[:6] == "FBUOY:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    if token[:6] == "FUBOY:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    if token[:6] == "FBOUY:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    if token[:6] == "GICA):":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    if token[:6] == "TBUOY:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:3] == "FS:":
        token = token[3:]
    elif token[:3] == "FB:":
        token = token[3:]
    elif token[:2] == "DS":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:2] == "G(":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:3] == "CA:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:3] == "GA:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:2] == "G:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]
    elif token[:2] == "M:":
        split = token.split(':')
        if len(split) > 1:
            token = token.split(':')[1]

    index = token.find("-2H")
    if index != -1:
        token = token[:index]
    index = token.find("-1H")
    if index != -1:
        token = token[:index]
    index = token.find("2-H")
    if index != -1:
        token = token[:index]
    index = token.find("1-H")
    if index != -1:
        token = token[:index]

    index = token.find("(")
    if index != -1:
        token = token[:index]
        
    return token

In [17]:
def create_dataset(directory, dataset_name):
    
    data_file = open(dataset_name, 'w')
    
    for filename in os.listdir(directory):
        if '.eaf' in filename:
            append_dataset(directory, filename, data_file)
            
    data_file.close()

def append_dataset(directory, eaf_file, data_file):
    eaf_object = pympi.Elan.Eaf(f'./{directory}/{eaf_file}')
    
    if not has_translation(eaf_object):
        return
    
    print(eaf_file)
    
    sentence, tokens = parse_eaf(eaf_object)
    
    tokens = filter_tokens(tokens)
    for i in range(0, len(sentence)):
        if len(tokens[i]) - len(sentence[i]) > 25:
            print(f'{sentence[i]}\t{tokens[i]}\n')
            continue
        data_file.write(f'{sentence[i]}\t{tokens[i]}\n')

def filter_tokens(tokens):
    
    filtered_tokens = []
    
    for token_sequence in tokens:
        filtered_token_sequence = []
        
        for token in token_sequence:
            
            if token[:3] == "PT:":
                continue
            if token[:5] == "LOOK(":
                continue
            if token[:7] == "PTBUOY:":
                continue
            
            token = get_token(token)
                    
            if token in ["WELL", "", "AHH", "UMM", "FSL", "PTBUOY"]:
                continue
            
            if token[-1].isdigit():
                token = token[:-1]
                
            if len(token) < 2:
                continue
            
            filtered_token_sequence.append(token)
        filtered_tokens.append(filtered_token_sequence)
                
    unduped_tokens = []
    for token_group in filtered_tokens:
        filtered_token_group = []
        for i in range(0, len(token_group)):
            if (i == 0 or token_group[i] != token_group[i - 1]) and (i < 2 or token_group[i] != token_group[i - 2]):
                filtered_token_group.append(token_group[i])
            
        unduped_tokens.append(join_tokens(filtered_token_group))
        
    return unduped_tokens

def has_translation(eaf_object):
    return eaf_object.tiers.get('FreeTransl') is not None

def parse_eaf(eaf_object):
    filtered_sentences = filter_tier(eaf_object, 'FreeTransl')
    filtered_lh = filter_tier(eaf_object, 'LH-IDgloss')
    filtered_rh = filter_tier(eaf_object, 'RH-IDgloss')
    
    lhp = 0
    rhp = 0
    tokens = []
    for sentence in filtered_sentences:
        token_group = []
        while (lhp < len(filtered_lh) and filtered_lh[lhp][1] <= sentence[1] + 2):
            token_group.append(filtered_lh[lhp])
            lhp += 1
        
        while (rhp < len(filtered_rh) and filtered_rh[rhp][1] <= sentence[1] + 2):
            token_group.append(filtered_rh[rhp])
            rhp += 1
        tokens.append(token_group)
    
    filtered_tokens = []
    for token_group in tokens:
        token_group.sort()
        filtered_token_group = []
        for i in range(0, len(token_group)):
            if i == 0 or token_group[i][2] != token_group[i - 1][2]:
                filtered_token_group.append(token_group[i][2])
            
        filtered_tokens.append(filtered_token_group)
    
    sentences = []
    for sentence in filtered_sentences:
        sentences.append(sentence[2])
        
    return sentences, filtered_tokens

def filter_tier(eaf_object, tier_name):
    segment_dict = eaf_object.tiers[tier_name][0]
    
    filtered_segments = []
    
    for key in segment_dict:
        segment = segment_dict[key]
        if segment[2] != '':
            filtered_segments.append((int(segment[0][2:]), int(segment[1][2:]), segment[2]))
        
    return filtered_segments
                        
def join_tokens(tokens):
    return " ".join(tokens)

In [18]:
directory = 'EAF'
dataset_name = 'elan_dataset.txt'

In [19]:
create_dataset(directory, dataset_name)

AAM1A1c2a.eaf
AAM1A2c6ii.eaf
AAM1A2c6iv.eaf
There on the counter, a phone was ringing.	G(ILY):UMM see TELEPHONE RINGING-PHONE-2H G(INDEFINITE):UMM FS:COUNTER DSS(5):COUNTER

A male receptionist frantically got to the phone and answered the phone. It looked as if he behaved obsequiously. 	MAN SLIDE1(BENT2) CA:MAN GRAB-2H TELEPHONE DSL(6):HOLDING-TELEPHONE FBUOY:TELEPHONE SAME PROUD TELEPHONE(FALSE-START) SAME SPIDER gaining someone's favour.

The cat held onto the birdcage and went down the apartment through the back door outside.	BIRD FS:CAGE(?) DSS(5):BIRDCAGE DSH(5):LIFT EXCITED DSM(1):DOWN DSG(B):HERE G(1):UMM BACK DOOR DSM(1):DOWN DSG(HCFLAT):FROM-HERE OUTSIDE

As the cat uncovers the birdcage,	DSM(FLATO):PRODS-AT-SOMETHING-IN THE -CAGE-WITH-HAND DSL(5):BIRDCAGE SHEET DSM(F):UNCOVER DSH(F):HOLDING-BIRDCAGE DSL(5):BIRDCAGE

AAM2A1c2b.eaf
There was a big hare, with big ears, prancing about.	RABBIT(H)-2H FS:HARE BIG DSS(1):HARE'S-EARS RABBIT-2H DSM(BENT2-DOWN):HARE-PRANCING

The big h

ADPA2c6ii.eaf
AFL2c7a.eaf
Then the both of them set off into the forest, calling out and looking high and low for the frog.	G(CA):PICKS-UP-COMFORTS-DOG BOTH YELL1-2H OUTSIDE FS:F(FALSE-START) DSS(4):LONG-VERTICAL-OBJECTS-EXTEND-FOREST FS:FORREST(ORREST) LOOK

The boy sees a hole in the ground. He looks down it, and a beaver jumps out of it and hits him on the nose.	G(5-WIGGLE):UMM SHORT BOY LOOK DSL(BC):ROUND-HOLE-IN-GROUND FS:BEAVER DSS(BENT5):ROUND-HOLE DSM(FLATBC):ANIMAL-JUMPS-OUT-AT-BOY G(CA):STROKE-PAT

AFLA1c2b.eaf
But the tortoise beat him there, crossing the finish line first.	TORTOISE DS(B):FEET-STEP ARRIVE G(5-WIGGLE):UMM DSL(B-LATERAL):FINISH-LINE DSM(B-LATERAL):ANIMAL-MOVE

AFLA2c6ii.eaf
AJBB1c2bAMBI.eaf
AJBB2c6iiiAMBI.eaf
AJNA1c2aLH.eaf
AJNA2c6iiLH.eaf
AJPB1c2b.eaf
AJPB2c6iii.eaf
As the man played music, a cat came out and picked up a banana.  	MUSIC G(1):UMM CAT2-2H DSM(BENT2):FROM-THERE-TO-HERE B(FALSE-START) BANANA DSM(S):GIVE-BANANA

AJPB2c7a.eaf
No, first he puts a fr

BCPA1c2a.eaf
From all OVER the place, they rush together and charge up the hill. A real mob, torches burning, waving pitchforks, with shovels from their farms, the works.	DS(BENT5):MANY-HUMANS-GATHER-EXCITED DSM(5-VERT):MANY-HUMANS-MOVE DS(5):FLAME DS(5):GROUND DS(6):SHAPE-OF-STICK DS(5):FLAME DS(3):PITCHFORK DS(BC):HOLDING-PITCHFORK G(5-UP):WELL DS(3):HOLDING-PITCHFORK DS(BC):HOLDING-PITCHFORK FULL SHOVEL DSM(5-VERT):MANY-HUMANS-MOVE

They rush up there, and find nothing. They don't get it, there's just the sheep, looking up from eating grass all quizzical at this crazy mob that's just standing there.	WHAT DS(5):PEOPLE NOTHING G(5-UP):WELL GIBBERISH SHEEP GRAZE DS(BO):ANIMAL-LOOKING DSM(BENT2):EAR-OF-SHEEP LOOK WHAT G(5-UP):WELL-WHAT? DS(5):SHEEP LOOK DS(6):HOLD-STICK WHAT DS(5):PEOPLE G(5-UP):WELL

As they start moving off, the boy bursts out laughing at how he fooled the lot of them, and got them to come running. He was having a ball.	DS(RUDE):MANY-HUMANS-SURGE-FORWARD DS(BENT5):MAN

MBHA3c9a.eaf
MCDB1c2a.eaf
MCDB2c6iii.eaf
MCDB2c7a.eaf
After a while, the boy gets into bed and falls sleep. His dog curls up beside him on the bed.	AND-THEN G(HOLD-PAUSE):UMM BOY BED GET-INTO-BED SLEEP BED DSM(H):HUMAN-GETS-INTO-BED WITH DOG DSL(BENT2):ANIMAL-BE-AT-BED DSS(B):FLAT-SURFACE-BED

The frog is also asleep.	FROG DSL(2):ANIMAL-BE-IN-JAR DSS(BENT5):SPHERICAL-JAR SLEEP

Later the frog hops out of the jar and takes off.	DSM(BENT2):ANIMAL-GETS-OUT-OF-JAR DSS(BENT5):SPHERICAL-JAR ESCAPE? DISAPPEAR

At the same time, the dog gets his head caught inside the jar. He moves about trying to get it off his head but instead he falls out the window and onto the ground below.	DOG LOOK SAME-TIME DOG DSS(BC):SPHERICAL-JAR DSM(S):ANIMAL-HEAD-INSIDE-BOWL DSL(BC):SPHERICAL-JAR-AROUND-HEAD DSM(S):ANIMAL-HEAD-IN-JAR DSS(BENT5):SPHERICAL-JAR G(CA):ANIMAL-FEET-MOVE DSM(BENT2):ANIMAL-FALLS DSS(B):FLAT-SURFACE-WINDOWSILL DSM(BENT2):ANIMAL-HITS-GROUND-HEAD-FIRST DSS(B):FLAT-SURFACE-GROUND

When he hits

MKB1B2c6iii.eaf
MKB1B2c7a.eaf
It was one of those big glass jars that you can fill with lollies or biscuits.	GLASS DSS(BENT5):SPHERICAL-JAR BEFORE LOLLY G(5-UP):OR BISCUIT DSS(BC):SPHERICAL-JAR PUT FILL DSS(BENT5):SPHERICAL-JAR DSS(B):HEIGHT-OF-JAR

It was the boy's pet dog.	G(HOLD-PAUSE):UMM HAVE FS:PET DOG G:I-EXPECT-YOU-TO-UNDERSTAND-THAT

Both were good and happy.	CHATTERBOX GOOD LAUGH DSH(BENT5):PLACE-SPHERICAL-JAR GOOD

Meanwhile, the little frog noticed they were asleep, and climbed up to the rim of the jar.	G:SLEEP-GOOD GLASS DSS(BENT5):SPHERICAL-JAR FROG SEE DSM(2):ANIMAL-WALKS-BACKWARD DSM(1):ANIMAL-MOVES-LEG-OVER DSM(BENT2):ANIMAL-GETS-OUT-OF-JAR DSS(BENT5):SPHERICAL-JAR DSH(S):HOLDS-THE-TOP-OF-JAR?

It put its head in the jar to check it and got stuck there.	DOG SEE DSS(BENT5):SPHERICAL-JAR-AROUND-HEAD WINDOW GLASS DSS(BC):SPHERICAL-JAR-AROUND-HEAD LOOK

It fell - with the glass jar still on its head - to the ground, and the jar smashed.	G(CA):DOG-LOSE-BALANCE WITH DSS(BC):

...

PDHA1c2a.eaf
Before they had come, thinking something terrible had happened	ALRIGHT STORY TITLE WHAT THE(SE) BOY CRY YELL2-2H WOLF TITLE WHAT STORY BOY LOOK-AFTER DOG(FALSE-START) SHEEP DOG NO-WAY SHEEP DSM(5-HORI):MANY-ANIMALS-MOVE WHERE IN COUNTRY AREA FS:VILLAGE DSS(5):VALLEY STOP AGAIN PUSH-ASIDE BOY LOOK-AFTER SHEEP DSM(5-HORI):MANY-ANIMALS-MOVE WHERE IN COUNTRY AREA FS:VILLAGE DSS(5):VALLEY WORK GO-POINT COLLECT SHEEP SEE PEOPLE HOUSE COLLECT ENCOURAGE DSM(5-HORI):MANY-ANIMALS-MOVE DSS(5-HORI):SLOPE ENCOURAGE STAND SHEEP GRAZE BOY SOMETIMES BORING LOVELY GOOD VIEW-2H LOOK G(5-UP):WELL TIME-WATCH FAST CONTINUE G(5-UP):WELL ONE REAL COINCIDENCE BORING SAME LOOK SHEEP GRAZE LOOK LATER(B) FROM MORNING NIGHT G(5-UP):WELL BORING LOOK G(5-UP):WELL ONE DAY IN COLD(FALSE-START) NO-WAY-2H IN ONE DAY THINK WHY-NOT SOME DIFFERENT SUPPOSE TEASE PEOPLE AREA SOME GOOD GO-TRACE ONE DAY PEOPLE DSM(5-HORI):MANY-HUMANS YELL2-2H WOLF WILL BITE SHEEP PEOPLE DSM(5-HORI):MANY-HUMANS-MOVE CRANKY TERRIBL

The dog went searching, put its head in the jar, straightened it and turned its head from side to side.	DOG SEARCH DSM(S):HEAD-GOES-DOWN DSL(BC):NECK-OF-JAR IN FS:JAR DSM(S):HEAD-GOES-DOWN-AND-UP DSL(BC):NECK-OF-JAR DSM(S):HEAD-TURNS-AROUND-INSIDE DSS(BC):SPHERE-JAR DSM(S):HEAD-TURNS-SIDE-TO-SIDE DSL(BC):NECK-OF-JAR

The dog sitting on the windowsill, moved its head from side to side, crouched and fell to the ground, smashing and scattering glass.	DOG DS(S):ANIMAL-TURNS-HIS-HEAD-SIDE-TO-SIDE-AND-DOWNWARD DSS(BENT5):SPHERICAL-JAR DS(2):ANIMAL-FALLS-DOWN DS(BBENT):THE-WINDOWSILL WINDOW DS(S):ANIMAL-FALLS-DOWN BREAK(5) BREAK FS:JAR

The boy was grumpy but he patted the dog.	BOY GRUMPY DS(BC):HUMAN-PATS-THE-DOG-ON-HIS-TRUNK DS(BC):HUMAN-CARRIES-THE-DOG ALRIGHT DS(BC):HUMAN-PATS-THE-DOG-ON-HIS-TRUNK

The dog looked and barked at bees as they swarmed near their dome-shaped hive.	DOG HOLE2-F LOOK BARK-FLATBC FS:BEES DS(5):THE-BEES-FLY BAD DS(BC):SHAPE-OF-SOMETHING-LIKE-A-HIVE HIVE DS(BC):SHAP

The turtle had already crossed the finish line.	DS(B):GROUND TORTOISE OVER-THE-LINE PAST DSM(B-HORI):ANIMAL-MOVES DSL(1-HORI):FINISH-LINE FINISH.GOOD

SBS1A2c9a.eaf
SBS1A3c7a.eaf
Before looking at the frog again, he and his dog went to bed and lay on the bed together.	BED LOOK GO BED DSM(HBENT):HUMAN-LIES-ON-BED DSS(B):FLAT-SURFACE-BED DOG SAUSAGE DSM(HBENT):ANIMAL-LIES-ON-BED DSS(B):FLAT-SURFACE-BED WITH-1H DSM(HBENT):HUMAN-LIES-ON-BED

When the bedroom was dark, the frog got out of the jar, jumped down from the window and was gone.	FROG DSL(BENT2-HORI):ANIMAL-SITS-IN-JAR DSS(BC):SPHERICAL-JAR NIGHT DSM(BENT2):ANIMAL-GETS-OUT-OF-JAR DSS(BC):SPHERICAL-JAR WINDOW DSM(BENT2):ANIMAL-JUMPS-OUT-OF-WINDOW DSS(B):VERTICAL-WINDOWSILL GONE1-2H

The dog put its head into the jar to see if the frog was inside but nothing was there.	DOG DSS(BENT5):SPHERICAL-JAR-AROUND-HEAD DSM(H):ANIMAL'S-HEAD-MOVES-INTO-THE-JAR DSS(BENT5):SPHERICAL-JAR DSS(BENT5):SPHERICAL-JAR-AROUND-HEAD DSM(H):ANIMAL'S-HEAD-MOV

SMCB2c6ii.eaf
There's a cat, and he gets into a drainpipe and crawls up it.	CAT DS(1):ANIMAL-GETS-INTO-SOMETHING DS(B):SOMETHING FS:GUTTER FS:PIPE FS:DOWNPIPE DS(1):ANIMAL-GETS-INTO-THE-OPENING DS(INDEFINITE):THE-DOWN-SPOUT DSM(BENT2):ANIMAL-CLIMBS-UP DS(1):ANIMAL-CLIMBS-UP

He goes off and gets a ball and drops it down the drainpipe.	DSM(BENT2):ANIMAL-GOES-SOMEWHERE DS(BENT5):ANIMAL-GETS-SOMETHING-LIKE-A-BALL BALL G(CA):CARRY-BIG-BALL GO-DOWN FS:PIPE

The ball goes right down the throat of the very surprised cat.	DS(BENT5):ANIMAL-DROPS-THE-BALL-INTO-THE-RAINHEAD GO-DOWN DS(INDEFINITE):THE-DOWN-SPOUT FS:TO CAT G(5-UP):WELL-NEVER-THOUGHT THINK DS(IRISHK):THE-BALL-GOES-INTO-THE-CAT'S-MOUTH SWALLOW DS(5):ANIMAL'S-CHEST

The cat's forced out of the pipe.	DS(1):ANIMAL-WITH-THE-BALL-GOES-DOWN DS(IRISHK):THE-ESOPHAGUS FS:OUT FS:OF(F) DS(GC):THE-DOWN-SPOUT

The cat, in pain because the ball is still in his tummy, rushes to a ten-pin bowling alley and goes in.	CAT SPRINT WITH BALL SWALLOW DS(B)

STBA2c6ii.eaf
There's a drainpipe coming down from the top of the building.	DSM(BENT2):ANIMAL-WALK SOMETHING-LIKE-A-POLE FS:POPE FS:DOWN WATER DSS(BC):VERTICAL-CYLINDER-DOWNPIPE RAIN DS(4):RAIN-FLOW-DOWN

He climbs into the opening and shimmies up the drainpipe.	DSS(BC):VERTICAL-CYLINDER-DOWNPIPE LOOK WANT DS(BC):THE-OPENING-OF-THE-DOWNPIPE DS(2):ANIMAL-GETS-INTO-THE-OPENING UP DS(BENT5):ANIMAL-CLIMBS-UP DSM(BENT2):ANIMAL-CLIMBS-UP DSS(BC):VERTICAL-CYLINDER-DOWNPIPE

He runs off and finds a bowling ball.	DSM(BENT2):ANIMAL DS(B):SOMETHING-LIKE-A-ROOF RUN- SEARCH FIND TEN BOWLING BALL

He throws the ball down the drainpipe.	G(CA):CARRY-BIG-BALL G(CA):DROP-BIG-BALL DSM(1-VERT):ENTITY-GO-DOWN DSS(BC):VERTICAL-CYLINDER-DOWNPIPE FS:P

As the cat climbs up, the bowling ball comes down, and it goes down the cat's throat!	DS(1):THE-BALL-CONTINUES-GOING-DOWN DSS(BC):VERTICAL-CYLINDER-DOWNPIPE DS(O)?:WITH-A-FAST-MOVEMENT-THE-BALL-GOES-DOWN CAT DS(BENT5):ANIMAL-CLIMBS-UP GO-POINT SWALLOW BAD DS(O)

In [20]:
eaf_object = pympi.Elan.Eaf(f'./{directory}/AAPB1c2b.eaf')
filtered_sentences = filter_tier(eaf_object, 'RH-IDgloss')

In [21]:
filtered_sentences

[(1, 8, 'G(5-UP):WELL'),
 (12, 16, 'PT:PRO2SG'),
 (20, 24, 'KNOW'),
 (28, 32, 'STORY'),
 (36, 40, 'RABBIT'),
 (44, 48, 'FS:RABBIT'),
 (52, 55, 'AND1'),
 (58, 62, 'FS:THE'),
 (66, 70, 'FS:TROTOISE'),
 (76, 80, 'TORTOISE'),
 (86, 91, 'CUTE'),
 (94, 98, 'ONE'),
 (105, 112, 'RABBIT'),
 (116, 120, 'FS:RABBIT'),
 (124, 128, 'PT:PRO3SG'),
 (132, 135, 'UGLY3'),
 (138, 142, 'BIG'),
 (146, 152, 'RABBIT'),
 (161, 165, 'ALWAYS1'),
 (169, 176, 'SHOW-OFF4?'),
 (183, 188, 'PT:POSS3SG'),
 (200, 204, 'PT:DET(7)'),
 (208, 212, 'TORTOISE'),
 (216, 220, 'DSS(B):ROUND-FLAT-OBJECT-LOCATED-AT'),
 (224, 229, 'CUTE'),
 (234, 240, 'LOOK-2H'),
 (248, 252, 'RABBIT'),
 (256, 261, 'SEE'),
 (264, 270, 'PT:PRO3SG-2H'),
 (277, 284, 'LAUGH'),
 (289, 297, 'PT:PRO2SG'),
 (306, 309, 'WHY-BECAUSE'),
 (312, 316, 'PT:PRO2SG'),
 (320, 324, 'TORTOISE(FALSE-START)'),
 (328, 334, 'SLOW'),
 (340, 349, 'TORTOISE'),
 (354, 361, 'DSM(S):FEET-STEP'),
 (369, 375, 'PT:LOC/PRO2SG'),
 (381, 388, 'G(CA):'),
 (391, 395, 'G(5-UP):WELL(7)'),